In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd

sp500_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_table = pd.read_html(sp500_url)
tikrs = sp500_table[0]['Symbol'].tolist()

market_cap=np.zeros(len(tikrs))
forward_pe=np.zeros(len(tikrs))
trailing_pe=np.zeros(len(tikrs))
price_to_book=np.zeros(len(tikrs))
price_to_sales=np.zeros(len(tikrs))
ev_ebitda=np.zeros(len(tikrs))
ev_profit=np.zeros(len(tikrs))

for i,t in enumerate(tikrs):
    stock_data=yf.Ticker(t).info
    market_cap[i] = stock_data.get('marketCap', np.nan)
    forward_pe[i] = stock_data.get('forwardPE', np.nan)
    trailing_pe[i] = stock_data.get('trailingPE', np.nan)
    price_to_book[i] = stock_data.get('priceToBook', np.nan)
    price_to_sales[i] = stock_data.get('priceToSalesTrailing12Months', np.nan)
    ev_ebitda[i] = stock_data.get('enterpriseToEbitda', np.nan)
    ev_profit[i] = stock_data.get('enterpriseToRevenue', np.nan) * 1/stock_data.get('profitMargins', np.nan)

metrics_df = pd.DataFrame({
    'Symbol': tikrs,
    'Market Cap': market_cap,
    'Forward P/E': forward_pe,
    'Trailing P/E': trailing_pe,
    'P/B': price_to_book,
    'P/S': price_to_sales,
    'EV/EBITDA': ev_ebitda,
    'EV/Profit': ev_profit
})
metrics_df.to_csv('metrics.csv', index=False)

In [ ]:
import pandas as pd

metrics_df=pd.read_csv('metrics.csv')

for col in metrics_df.columns[metrics_df.isnull().any()]:
    if pd.api.types.is_numeric_dtype(metrics_df[col]):
        metrics_df[col] = metrics_df[col].fillna(metrics_df[col].median())

metrics_df = metrics_df[metrics_df['Forward P/E'] >= 0]
metrics_df = metrics_df[metrics_df['Trailing P/E'] >= 0]
metrics_df=metrics_df[metrics_df['Trailing P/E']!=np.inf]
metrics_df['pe_growth']=metrics_df['Trailing P/E']-metrics_df['Forward P/E']
metrics_df=metrics_df.sort_values(by='Market Cap',ascending=False).reset_index(drop=True)
metrics_df.to_csv('metrics_clean.csv', index=False)

In [21]:
import pandas as pd

metrics_df=pd.read_csv('metrics_clean.csv')

# Normalize the data in each column to the range [0, 1]
normalised_metric=metrics_df.copy()
for col in normalised_metric.columns[1:]: # Exclude 'Symbol
    if pd.api.types.is_numeric_dtype(normalised_metric[col]):
        min_val = normalised_metric[col].min()
        max_val = normalised_metric[col].max()
        if max_val != min_val: # prevent division by zero if min and max are equal
            normalised_metric[col] = (normalised_metric[col] - min_val) / (max_val - min_val)
        else:
            normalised_metric[col] = 0  # Or handle the case as needed, for example set all to 0 or 0.5

normalised_metric.drop(columns=['Market Cap','Forward P/E','Trailing P/E'],inplace=True)
metrics_df['score']=np.mean(normalised_metric.iloc[:,4:],axis=1)
metrics_df.sort_values(by='score',ascending=False)

,Symbol,Market Cap,Forward P/E,Trailing P/E,P/B,P/S,EV/EBITDA,EV/Profit,pe_growth,score
87,GILD,1.107188e+11,11.747110,987.112240,5.991779,3.912462,9.145,1013.033708,975.365130,1.000000
442,INCY,1.352403e+10,11.544154,779.999940,4.267996,3.318080,153.437,380.175659,768.455786,0.713140
107,CRWD,8.583941e+10,79.351906,697.000200,36.584095,24.412153,459.175,505.867769,617.648294,0.689951
291,IRM,2.964240e+10,51.471375,280.583300,3.508025,4.949921,23.166,443.020595,229.111925,0.507745
229,GLW,4.023326e+10,20.334423,247.315800,3.445015,3.190331,18.396,307.437348,226.981377,0.463915
...,...,...,...,...,...,...,...,...,...,...
499,CZR,6.665498e+09,25.242609,24.922298,1.591739,0.591385,8.854,-89.978145,-0.320311,0.243103
90,VRTX,1.056036e+11,21.915998,24.922298,6.761617,9.938417,22.799,-211.317843,3.006300,0.206063
80,BA,1.288612e+11,246.513140,24.922298,3.508025,1.758165,-38.430,-19.235366,-221.590842,0.173077
374,LUV,1.982131e+10,19.752214,24.922298,1.903912,0.724040,11.553,-405.714309,5.170084,0.145401


In [ ]:
stock_data['profitMargins']

In [ ]:
metrics_df

In [ ]:

'symbol'
'marketCap'
'forwardPE',-'trailingPE',
'priceToBook',
'priceToSalesTrailing12Months',
'enterpriseToEbitda',
'enterpriseToRevenue'*1/'profitMargins'

In [ ]:
dict_keys(['auditRisk', 'boardRisk', 'compensationRisk', 'shareHolderRightsRisk', 'overallRisk',  'priceHint', 'beta', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh',  'fiftyDayAverage', 'twoHundredDayAverage', 'trailingAnnualDividendRate', 'trailingAnnualDividendYield', 'currency', 'enterpriseValue', 'profitMargins', 'floatShares', 'sharesOutstanding', 'sharesShort', 'sharesShortPriorMonth', 'sharesShortPreviousMonthDate', 'dateShortInterest', 'sharesPercentSharesOut', 'heldPercentInsiders', 'heldPercentInstitutions', 'shortRatio', 'shortPercentOfFloat', 'impliedSharesOutstanding', 'bookValue',  'lastFiscalYearEnd', 'nextFiscalYearEnd', 'mostRecentQuarter', 'earningsQuarterlyGrowth', 'netIncomeToCommon', 'trailingEps', 'forwardEps', 'enterpriseToRevenue',  '52WeekChange', 'SandP52WeekChange', 'currentPrice', 'targetHighPrice', 'targetLowPrice', 'targetMeanPrice', 'targetMedianPrice', 'recommendationMean', 'recommendationKey', 'numberOfAnalystOpinions', 'totalCash', 'totalCashPerShare', 'ebitda', 'totalDebt', 'quickRatio', 'currentRatio', 'totalRevenue', 'debtToEquity', 'revenuePerShare', 'returnOnAssets', 'returnOnEquity', 'freeCashflow', 'operatingCashflow', 'earningsGrowth', 'revenueGrowth', 'grossMargins', 'ebitdaMargins', 'operatingMargins',  'trailingPegRatio'])




